In [7]:
from dotenv import load_dotenv
load_dotenv("/home/longview/Dev/.env")

True

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

'Why did the ice cream truck break down?\n\nBecause it had too many "scoops"!'

In [10]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [11]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [12]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

In [13]:
message = model.invoke(prompt_value)
message

AIMessage(content='Why did the ice cream truck break down? \nBecause it had too many "scoops"!', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 15, 'total_tokens': 36}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-d81d0e2c-51cd-4ede-9536-b6891358834f-0')

In [14]:
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\n\nRobot: Why did the ice cream go to therapy? Because it was feeling a little rocky road!'

In [15]:
output_parser.invoke(message)

'Why did the ice cream truck break down? \nBecause it had too many "scoops"!'

In [16]:
for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

Why did the ice cream break up with the sundae? It just couldn't handle the toppings anymore!

In [20]:
stream_cache = []
for chunk in chain.stream("ice cream"):
    stream_cache.append(chunk)
    print(chunk, end="", flush=True)

Why did the ice cream cone go to therapy? Because it had too many sprinkles of anxiety!

In [21]:
stream_cache

['',
 'Why',
 ' did',
 ' the',
 ' ice',
 ' cream',
 ' cone',
 ' go',
 ' to',
 ' therapy',
 '?',
 ' Because',
 ' it',
 ' had',
 ' too',
 ' many',
 ' sprink',
 'les',
 ' of',
 ' anxiety',
 '!',
 '']

In [22]:
stream_cache = []
for chunk in chain.stream("ice cream"):
    stream_cache.append(chunk)
    print(chunk, end="")

Why did the ice cream truck break down? It just couldn't handle the rocky road!

In [23]:
stream_cache

['',
 'Why',
 ' did',
 ' the',
 ' ice',
 ' cream',
 ' truck',
 ' break',
 ' down',
 '?',
 ' It',
 ' just',
 ' couldn',
 "'t",
 ' handle',
 ' the',
 ' rocky',
 ' road',
 '!',
 '']

## RAG Search Example

In [1]:
from dotenv import load_dotenv
load_dotenv("/home/longview/Dev/.env")

True

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [6]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever()

template = """Anser the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("Where did harrison work?")

/home/longview/Dev/langchain_playground/.venv/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


'Harrison worked at Kensho.'

## I/O Exploration

In [24]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama

llm = Ollama(model="llama2")
chat_model = ChatOllama()

In [25]:
llm.invoke("Hello, how are you today?")

"\nI'm just an AI, I don't have feelings or emotions like humans do, so I can't really feel or respond to emotions the way a human would. However, I'm here to help you with any questions or tasks you may have! How can I assist you today?"

In [26]:
chat_model.invoke("Hello, how are you today?")

AIMessage(content="\nI'm just an AI, I don't have emotions or feelings, so I can't experience the world in the same way that humans do. However, I'm here to help you with any questions or tasks you may have! Is there something specific you'd like to know or discuss?", response_metadata={'model': 'llama2', 'created_at': '2024-04-23T23:15:24.810818362Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 2058486152, 'load_duration': 4059124, 'prompt_eval_duration': 400961000, 'eval_count': 67, 'eval_duration': 1652886000}, id='run-1b991ff7-466a-462f-8619-83f97401c689-0')

Force unload the model from vram

In [27]:
! curl http://localhost:11434/api/generate -d '{"model": "llama2", "keep_alive":0}'

{"model":"llama2","created_at":"2024-04-23T23:18:32.642245954Z","response":"","done":true}